```
Copyright 2024 Control-M Jupyter Notebooks Project Contributors

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
```

**Author:** Control-M Solution Engineering Team  
**E-Mail:** orchestrator@bmc.com  
**Version:** 1.0.0  
**License:** Apache 2.0  

**Warranty Disclaimer:** This software is provided "AS IS" without warranty of any kind. Use at your own risk.

# Control-M Environment Configuration

Interactive notebook to create and update the `.env` file for Control-M API access.

**Instructions:** Run all cells in order, then use the interactive configuration controls at the bottom.

In [1]:
import os
import ipywidgets as widgets
from IPython.display import display, clear_output
import base64
import re
import requests
from typing import Dict, Tuple, Any

# Widget variable declarations for type checking
api_token_widget: widgets.Text
hostname_widget: widgets.Text
port_widget: widgets.Text
validation_output: widgets.Output

In [2]:
# Configuration file path
ENV_FILE_PATH = '.env'

def load_current_env() -> Dict[str, str]:
    """Load current environment variables from .env file"""
    env_vars = {
        'WERKSTATT_CTM_AAPI': '',
        'WERKSTATT_CTM_HOST': '',
        'WERKSTATT_CTM_PORT': '443'
    }
    
    try:
        with open(ENV_FILE_PATH, 'r') as f:
            for line in f:
                line = line.strip()
                if line and not line.startswith('#') and '=' in line:
                    key, value = line.split('=', 1)
                    key = key.strip()
                    value = value.strip().strip('"').strip("'")
                    if key in env_vars:
                        env_vars[key] = value
        print(f"✅ Loaded existing configuration from: {ENV_FILE_PATH}")
    except FileNotFoundError:
        print(f"📝 No existing .env file found. Will create new one at: {ENV_FILE_PATH}")
    except Exception as e:
        print(f"⚠️ Error reading .env file: {e}")
    
    return env_vars

# Load current environment
current_env = load_current_env()
print("\n📋 Current Configuration:")
for key, value in current_env.items():
    display_value = value if value else "(not set)"
    if key == 'WERKSTATT_CTM_AAPI' and value:
        display_value = f"{value[:10]}...{value[-10:]}" if len(value) > 20 else value
    print(f"  {key}: {display_value}")

print("\n⬇️ Run all cells below first, then use the configuration interface at the bottom!")

✅ Loaded existing configuration from: .env

📋 Current Configuration:
  WERKSTATT_CTM_AAPI: b25QcmVtOj...c4ZTMwNQ==
  WERKSTATT_CTM_HOST: ctm.shytwr.net
  WERKSTATT_CTM_PORT: 443

⬇️ Run all cells below first, then use the configuration interface at the bottom!


In [3]:
# Validation functions
def validate_api_token(token: str) -> Tuple[bool, str]:
    """Basic validation for API token"""
    if not token:
        return False, "API token cannot be empty"
    
    # Check if it's base64 encoded (basic check)
    try:
        if len(token) > 10:  # Reasonable minimum length
            base64.b64decode(token)
            return True, "Valid base64 encoded token"
    except Exception:
        pass
    
    if len(token) > 10:  # Accept any reasonable length token
        return True, "Token format accepted"
    
    return False, "Token appears too short"

def validate_hostname(host: str) -> Tuple[bool, str]:
    """Basic validation for hostname"""
    if not host:
        return False, "Hostname cannot be empty"
    
    # Basic hostname pattern
    if re.match(r'^[a-zA-Z0-9.-]+$', host) and '.' in host:
        return True, "Valid hostname format"
    
    return False, "Invalid hostname format (should be like: ctm.example.com)"

def validate_port(port: str) -> Tuple[bool, str]:
    """Basic validation for port number"""
    if not port:
        return False, "Port cannot be empty"
    
    try:
        port_num = int(port)
        if 1 <= port_num <= 65535:
            return True, f"Valid port number: {port_num}"
        else:
            return False, "Port must be between 1 and 65535"
    except ValueError:
        return False, "Port must be a number"

print("🔧 Validation functions defined and ready!")

🔧 Validation functions defined and ready!


In [4]:
# Configuration management functions
def validate_all_fields() -> bool:
    """Validate all configuration fields"""
    with validation_output:
        clear_output()
        
        print("🔍 Validating configuration...\n")
        
        # Validate API token
        token_valid, token_msg = validate_api_token(api_token_widget.value.strip())
        token_icon = "✅" if token_valid else "❌"
        print(f"{token_icon} API Token: {token_msg}")
        
        # Validate hostname
        host_valid, host_msg = validate_hostname(hostname_widget.value.strip())
        host_icon = "✅" if host_valid else "❌"
        print(f"{host_icon} Hostname: {host_msg}")
        
        # Validate port
        port_valid, port_msg = validate_port(port_widget.value.strip())
        port_icon = "✅" if port_valid else "❌"
        print(f"{port_icon} Port: {port_msg}")
        
        return token_valid and host_valid and port_valid

def save_configuration(b: widgets.Button) -> None:
    """Save configuration to .env file"""
    with validation_output:
        clear_output()
        
        # Validate first
        if not validate_all_fields():
            print("\n❌ Please fix validation errors before saving.")
            return
        
        try:
            # Prepare the .env content
            env_content = f"""# Control-M API Configuration
# Generated by ctm-config.ipynb

WERKSTATT_CTM_AAPI={api_token_widget.value.strip()}
WERKSTATT_CTM_HOST={hostname_widget.value.strip()}
WERKSTATT_CTM_PORT={port_widget.value.strip()}
"""
            
            # Write to file
            with open(ENV_FILE_PATH, 'w') as f:
                f.write(env_content)
            
            print(f"\n✅ Configuration saved successfully to: {ENV_FILE_PATH}")
            print("\n📋 Saved Configuration:")
            print(f"  WERKSTATT_CTM_HOST: {hostname_widget.value.strip()}")
            print(f"  WERKSTATT_CTM_PORT: {port_widget.value.strip()}")
            api_display = api_token_widget.value.strip()
            if len(api_display) > 20:
                api_display = f"{api_display[:10]}...{api_display[-10:]}"
            print(f"  WERKSTATT_CTM_AAPI: {api_display}")
            
            print("\n🚀 Ready to use other notebooks! Run ctm.ipynb next to collect inventory.")
            
        except Exception as e:
            print(f"\n❌ Error saving configuration: {e}")

def test_connection(b: widgets.Button) -> None:
    """Test connection to Control-M API"""
    with validation_output:
        clear_output()
        
        # Validate first
        if not validate_all_fields():
            print("\n❌ Please fix validation errors before testing.")
            return
        
        print("🔍 Testing connection to Control-M API...\n")
        
        try:
            import requests
            
            # Build URL and headers
            base_url = f"https://{hostname_widget.value.strip()}:{port_widget.value.strip()}/automation-api"
            headers = {
                "x-api-key": api_token_widget.value.strip(),
                "Content-Type": "application/json",
                "Accept": "application/json"
            }
            
            print(f"📡 Testing URL: {base_url}/config/servers")
            
            # Test with a simple API call
            response = requests.get(f"{base_url}/config/servers", headers=headers, verify=True, timeout=30)
            
            if response.status_code == 200:
                servers = response.json()
                print(f"✅ Connection successful!")
                print(f"📊 Found {len(servers)} Control-M servers")
                if servers:
                    print("\n📋 Available servers:")
                    for server in servers[:5]:  # Show first 5
                        server_name = server.get('name', 'Unknown')
                        server_state = server.get('state', 'Unknown')
                        print(f"  • {server_name}: {server_state}")
                    if len(servers) > 5:
                        print(f"  ... and {len(servers) - 5} more servers")
                
                print("\n🎉 Configuration is working! You can now use the other notebooks.")
                
            else:
                print(f"❌ Connection failed: HTTP {response.status_code}")
                print(f"Response: {response.text[:200]}..." if len(response.text) > 200 else response.text)
                
        except requests.exceptions.RequestException as e:
            print(f"❌ Connection error: {e}")
            print("\n💡 Common issues:")
            print("  • Check if hostname is correct and reachable")
            print("  • Verify port number (usually 443 or 8443)")
            print("  • Ensure API token is valid and not expired")
            print("  • Check network connectivity and firewall settings")
        except Exception as e:
            print(f"❌ Unexpected error: {e}")

print("🎯 Configuration functions defined and ready!")

🎯 Configuration functions defined and ready!


---
# 🎛️ Interactive Configuration Interface

**Make sure all cells above have been executed successfully before using these controls!**

## 📝 Configuration Fields

**Instructions:**
- **API Token**: Base64 encoded authentication token from Control-M
- **Hostname**: Your Control-M Enterprise Manager hostname (e.g., `ctm.company.com`)
- **Port**: HTTPS port for Control-M API (usually `443` or `8443`)

In [5]:
# Create interactive widgets for configuration - REQUIRES ALL CELLS ABOVE TO BE RUN FIRST
if 'current_env' in globals():
    # Create input widgets - API token now uses single-line Text widget
    api_token_widget = widgets.Text(
        value=current_env['WERKSTATT_CTM_AAPI'],
        placeholder='b25QcmVtOjA0Y2NkMjYyLTBhY2QtNGFlZS04YWFmLTMzNWIxMjc4ZTMwNQ==',
        description='API Token:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='600px')
    )
    
    hostname_widget = widgets.Text(
        value=current_env['WERKSTATT_CTM_HOST'],
        placeholder='ctm.example.com',
        description='Hostname:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='400px')
    )
    
    port_widget = widgets.Text(
        value=current_env['WERKSTATT_CTM_PORT'],
        placeholder='443',
        description='Port:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='100px')
    )
    
    # Save button (appears first)
    save_button = widgets.Button(
        description='💾 Save Configuration',
        button_style='success',
        layout=widgets.Layout(width='200px')
    )
    
    # Test connection button (appears last)
    test_button = widgets.Button(
        description='🔍 Test Connection',
        button_style='info',
        layout=widgets.Layout(width='200px')
    )
    
    # Validation output
    validation_output = widgets.Output()
    
    # Connect button handlers
    save_button.on_click(save_configuration)
    test_button.on_click(test_connection)
    
    print("🎛️ Configuration Interface Ready:")
    display(widgets.VBox([
        widgets.HTML("<h3>Control-M API Configuration</h3>"),
        api_token_widget,
        hostname_widget,
        port_widget,
        widgets.HTML("<p><strong>Step 1:</strong> Save configuration first, then test connection</p>"),
        widgets.HBox([save_button, test_button]),
        validation_output
    ]))
    
    print("\n📝 Usage Instructions:")
    print("1. Fill in your Control-M API configuration above")
    print("2. Click 'Save Configuration' to create/update .env file")
    print("3. Click 'Test Connection' to verify settings")
    print("4. Run ctm.ipynb to collect inventory data")
    
else:
    print("⚠️ Please run all cells above first to load the configuration functions!")

🎛️ Configuration Interface Ready:



📝 Usage Instructions:
1. Fill in your Control-M API configuration above
2. Click 'Save Configuration' to create/update .env file
3. Click 'Test Connection' to verify settings
4. Run ctm.ipynb to collect inventory data
